In [3]:
import os.path as osp

In [4]:
# 학습 및 검증용 이미지 데이터, 어노테이션 데이터의 파일 경로 리스트 작성

def make_datapath_list(rootpath):
    """
    데이터 경로를 저장한 리스트 생성

    파라미터
    ---
    rootpath : str
        데이터 폴더 경로
    
    returns
    ---
    ret : train_img_list, train_anno_list, val_img_list, val_anno_list
        데이터 경로를 저장한 리스트
    """
    

    # 이미지 데이터와 어노테이션 파일의 경로 템플릿 작성
    imgpath_template = osp.join(rootpath, 'JPEGImages', '%s.jpg')
    annopath_template = osp.join(rootpath, 'Annotations', '%s.xml')

    # 학습 및 검증 파일 ID 휙득
    train_id_names = osp.join(rootpath + 'ImageSets/Main/train.txt')
    val_id_names = osp.join(rootpath + 'ImageSets/Main/val.txt')

    # 학습 데이터의 이미지 파일과 어노테이션 파일의 경로 리스트 작성
    train_img_list = list()
    train_anno_list = list()

    for line in open(train_id_names):
        file_id = line.strip()  # 공백과 줄 바꿈 제거
        img_path = (imgpath_template % file_id)  # 이미지 경로
        anno_path = (annopath_template % file_id)  # 어노테이션 경로
        train_img_list.append(img_path)  # 리스트에 추가
        train_anno_list.append(anno_path)  # 리스트에 추가

    # 검증 데이터의 이미지 파일과 어노테이션 파일의 경로 리스트 작성
    val_img_list = list()
    val_anno_list = list()

    for line in open(val_id_names):
        file_id = line.strip()  # 공백과 줄 바꿈 제거
        img_path = (imgpath_template % file_id)  # 이미지 경로
        anno_path = (annopath_template % file_id)  # 어노테이션 경로
        val_img_list.append(img_path)  # 리스트에 추가
        val_anno_list.append(anno_path)  # 리스트에 추가

    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [5]:
# 테스트
rootpath = './data/VOCdevkit/VOC2012/'
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

print(train_img_list[0])
     

FileNotFoundError: [Errno 2] No such file or directory: './data/VOCdevkit/VOC2012/ImageSets/Main/train.txt'

In [ ]:
# XML 형식의 어노테이션을 리스트 형식으로 변환하는 클래스

class Anno_xml2list(object):
    """
    한 이미지의 XM 형식 어노테이션 데이터를 이미지 크기로 규격화하여 리스트 형식으로 변환

    attributes
    ---
    classes : 리스트
        VOC의 클래스명을 저장한 리스트
    
    """

    def __init__(self, classes):

        self.classes = classes

    def __call__(self, xml_path, width, height):
        """
        한 이미지의 XML 형식 어노테이션 데이터를 이미지 크기로 규격화하여 리스트 형식으로 변환 

        파라미터
        ----------
        xml_path : str
            xml 파일 경로
        width : int
            대상 이미지 폭
        height : int
            대상 이미지 높이

        Returns
        -------
        ret : [[xmin, ymin, xmax, ymax, label_ind], ... ]
            물체의 어노테이션 데이터를 저장한 리스트. 이미지에 존재하는 물체 수만큼의 요소를 가진다.
        """

        # 이미지 내 모든 물체의 어노테이션을 이 리스트에 저장
        ret = []

        # xml 파일 로드
        xml = ET.parse(xml_path).getroot()

        # 이미지 내 object 수만큼 반복
        for obj in xml.iter('object'):

            # 어노테이션에서 검지가 difficult로 설정된 것은 제외
            difficult = int(obj.find('difficult').text)
            if difficult == 1:
                continue

            # 한 물체의 어노테이션을 저장하는 리스트
            bndbox = []

            name = obj.find('name').text.lower().strip()  #  물체 이름
            bbox = obj.find('bndbox')  # 바운딩 박스 정보

            # 어노테이션의 xmin, ymin, xmax, ymax를 취득하고 0~1로 정규화
            pts = ['xmin', 'ymin', 'xmax', 'ymax']

            for pt in (pts):
                # VOC데이터는 원점이 (1,1) 이므로 1을 빼서 (0,0) 변환한다.
                cur_pixel = int(bbox.find(pt).text) - 1

                # 폭, 높이로 규격화
                if pt == 'xmin' or pt == 'xmax':  # x 방향의 경우 폭으로 나눈다.
                    cur_pixel /= width
                else:  # y 방향의 경우 높이로 나눈다.
                    cur_pixel /= height

                bndbox.append(cur_pixel)

            # 어노테이션 클래스명 index를 츼득하여 추가
            label_idx = self.classes.index(name)
            bndbox.append(label_idx)

            #  res에 [xmin, ymin, xmax, label_ind]를 더한다
            ret += [bndbox]

        return np.array(ret)  # [[xmin, ymin, xmax, ymax, label_ind], ... ]

In [ ]:
# 테스트
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

transform_anno = Anno_xml2list(voc_classes)

# 이미지 로드용으로 OpenCV tkdyd
ind = 1
image_file_path = val_img_list[ind]
img = cv2.imread(image_file_path)  # 높이/폭/색RGB
height, width, channels = img.shape  # 이미지 shape

# 어노테이션을 리스트로 표시
transform_anno(val_anno_list[ind], width, height)

# 이미지와 어노테이션의 전처리를 실행하는 DataTransform 클래스 
- 
데이터 transform 적용 시 이미지를 확장하는데, 그때 BBox정도를 동시에 환형해야함.- 

이미지 데이터를 불때올 떄 OpenCV를 사용하는데 OpenCv로 이미지를 불러올때는 높이/폭/색상(BGR)순으로 불러온온다.

In [6]:

# 입력 이미지의 전처리 클래스
from utils.data_augumentation import Compose, ConvertFromInts, ToAbsoluteCoords, PhotometricDistort, Expand, RandomSampleCrop, RandomMirror, ToPercentCoords, Resize, SubtractMeans

class DataTransform():
    """
    이미지와 어노테이션의 전처리 클래스. 학습과 추론에서 다르게 작동한다.
    이미지 크기를 300x300으로 한다.
    학습 시에만 데이터의 이미지를 확장한다.

    attributes
    ---
    input_size : int
        리사이즈 대상 화상의 크기
    color_mean : (B, G, R)
        각 색상 채널의 평균 값
    """

    def __init__(self, input_size, color_mean):
        self.data_transform = {
            'train': Compose([
                ConvertFromInts(),  # int를 float32로 변환
                ToAbsoluteCoords(),  # 어노테이션 데이터의 규격화 반환
                PhotometricDistort(),  # 이미지의 색조 등 임의의 변화
                Expand(color_mean),  # 이미지의 캔버스 확대
                RandomSampleCrop(),  # 이미지내의 특정 부분 무작위 추출
                RandomMirror(),  # 이미지 반전
                ToPercentCoords(),  # 어노테이션 데이터를 0~1로 규격화
                Resize(input_size),  # 이미지 크기를 input_size x input_size로 변형
                SubtractMeans(color_mean)  # BGR 색상의 평균값 빼기
            ]),
            'val': Compose([
                ConvertFromInts(),  # int를 float으로 변환
                Resize(input_size),  # 화상 크기를 input_size x input_size로 변환
                SubtractMeans(color_mean)  # BGR 색상의 평균값 빼기
            ])
        }

    def __call__(self, img, phase, boxes, labels):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            전처리 모드 지정
        """
        return self.data_transform[phase](img, boxes, labels)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# 동작 확인 

# 1. 이미지 읽기
image_file_path = train_img_list[0]
img = cv2.imread(image_file_path)  # [높이/폭/색BGR]
height, width, channels = img.shape  # 이미지 크기 휙득

# 2. 어노테이션을 리스트로
transform_anno = Anno_xml2list(voc_classes)
anno_list = transform_anno(train_anno_list[0], width, height)

# 3. 원본 표시
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

# 4. 전처리 클래스 작성
color_mean = (104, 117, 123)  # BGR 색상의 평균 값
input_size = 300  # 화상의 input 사이즈를 300 x 300 으로
transform = DataTransform(input_size, color_mean)

# 5. train 이미지 변환 후 출력
phase = "train"
img_transformed, boxes, labels = transform(
    img, phase, anno_list[:, :4], anno_list[:, 4])
plt.imshow(cv2.cvtColor(img_transformed, cv2.COLOR_BGR2RGB))
plt.show()


# 6. val 이미지 변환 후 출력
phase = "val"
img_transformed, boxes, labels = transform(
    img, phase, anno_list[:, :4], anno_list[:, 4])
plt.imshow(cv2.cvtColor(img_transformed, cv2.COLOR_BGR2RGB))
plt.show()

# dataset 작성

In [ ]:


class VOCDataset(data.Dataset):
    """
    VOC2012의 Dataset을 만드는 클래스. 파이토치의 Dataset 클래스를 상속한다.

    Attributes
    ---
    img_list : 리스트
        이미지 경로를 저장한 리스트
    anno_list : 리스트
        어노테이션 겨ㅇ로를 저장한 리스트
    phase : 'train' or 'test'
        학습 또는 훈련 설정
    transform : object
        전처리 클래스의 인스턴스
    transform_anno : object
        xml 어노테이션을 리스트로 변환하는 인스턴스

    """

    def __init__(self, img_list, anno_list, phase, transform, transform_anno):
        self.img_list = img_list
        self.anno_list = anno_list
        self.phase = phase  # train or val
        self.transform = transform  # 이미지 변환
        self.transform_anno = transform_anno  # 어노테이션 데이터를 xml에서 리스트로 변경

    def __len__(self):
        '''이미지 갯수 반환'''
        return len(self.img_list)

    def __getitem__(self, index):
        '''
        전처리된 이미지의 텐서 형식 데이터와 어노테이션 반환
        '''
        im, gt, h, w = self.pull_item(index)
        return im, gt

    def pull_item(self, index):
        '''
        전처리한 이미지의 텐서 형식 데이터, 어노테이션, 이미지 높이, 폭 반환
        '''

        # 1. 이미지 로딩
        image_file_path = self.img_list[index]
        img = cv2.imread(image_file_path)  # 높이/넓이/색BGR
        height, width, channels = img.shape  # 이미지 shape

        # 2. xml 형식의 어노테이션 정보를 리스트에 저장
        anno_file_path = self.anno_list[index]
        anno_list = self.transform_anno(anno_file_path, width, height)

        # 3. 전처리 실시
        img, boxes, labels = self.transform(
            img, self.phase, anno_list[:, :4], anno_list[:, 4])

        # 색상 채널의 순서가 BGR이므로 RGB로 변경
        # 높이/ 폭/ 색상채널의 순서를 색상채널/높이/폭으로 변경
        img = torch.from_numpy(img[:, :, (2, 1, 0)]).permute(2, 0, 1)

        # BBox와 라벨을 세트로 한 np.array를 생성. 변수 이름 gt는 ground truth의 약칭
        gt = np.hstack((boxes, np.expand_dims(labels, axis=1)))

        return img, gt, height, width

In [ ]:

# 데이터셋 작동 확인
color_mean = (104, 117, 123)  # (BGR)의 색의 평균값
input_size = 300  # 이미지 input 사이즈를 300x300으로 한다.

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# 이미지 출력 예시
val_dataset.__getitem__(1)

## DataLoader 구현

object detection에서는 Dataloader를 조금 다르게 선언해야 한다.  
이미지 데이터마다 dataset에서 꺼낼 어노테이션 데이터 정보, gt 변수의 크기(화상 내의 물체 수)가 다르다. gt는 리스트형 변수이고 요소 수는 이미지 속 물체 수 이다.  
 각 요소는다섯 개의 변수 [xmin, ymin, xmax, ymax, class_index] 이다   .
dataset에서 꺼내는 변수의 크기가 데이터마다 다르다면 DataLoader 클래스에서 기본적으로 사용하는 데이터 추출 함수인 collate_fn을 별도로 만들어야 한다.

In [ ]:
def od_collate_fn(batch):
    """
    dataset에서 꺼내는 어노테이션 데이터의 크기는 이미지마다 다르다.
    이미지 내의 물체 수가 두 개이면 (2,5) 사이즈이지만 세 개이면 (3, 5)로 바뀐다.
    변화에 대응하는 DataLoader를 만드는 collate_fn을 작성한다.
    collate_fn은 파이토치 리스토로 mini-batch를 작성하는 함수이다. 
    미니 배치 분량의 화상이 나열된 리스트 변수 batch에 미니 배치 번호를 지정하는 차원을
    가장 앞에 하나 추가하여 리스트 형태로 변형한다.
    """

    targets = []
    imgs = []
    for sample in batch:
        imgs.append(sample[0]) # sample[0]는 이미지
        targets.append(torch.FloatTensor(sample[1])) # sample[1]은 어노테이션 gt
    
    # imgs는 미니 배치 크기의 리스트
    # 리스트 요소는 torch.Size([3, 300, 300])
    # 이 리스트를 torch.Size([batch_num,3,300,300])의 텐서로 변환
    imgs = torch.stack(imgs, dim = 0)

    # tragets는 어노테이션의 정답인 gt 리스트
    # 리스트 크기 = 미니 배치 크기
    # targets 리스트의 요소는 [n,5]
    # n은 이미지 마다 다르며 이미지 속 물체의 수
    # 5는 [xmin, ymin, xmax, ymax, class_index]

    return imgs, targets
     

In [ ]:
# 데이터 로더 테스트

batch_size = 4

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# dict 형식으로 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 동작 확인
batch_iterator = iter(dataloaders_dict["val"])  
images, targets = next(batch_iterator)  # 첫 번째 요소 추출
print(images.size())  # torch.Size([4, 3, 300, 300])
print(len(targets))
print(targets[1].size())  # 미니 배치 크기의 리스트, 각 요소는 [n,5], n은 물체 수
     